<a href="https://colab.research.google.com/github/MbohBless/Playing_with_groq/blob/main/NL2SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install -q groq
%pip install -q duckdb
%pip install -q sqlparse
%pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.4 M

In [3]:
import os
import duckdb
import groq
import sqlparse
import json
from typing import Dict, Any, List
import logging
import pandas as pd
from pathlib import Path
import gradio as gr
import shutil


In [3]:
%%writefile requirements.txt
groq
duckdb
sqlparse
pandas


Writing requirements.txt


In [4]:
# configuting logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:

def chat_with_groq(client:groq.Groq,
                   prompt:str,
                   model:str,
                   response_format: Dict[str, str]
                   )-> Any:
                   """
                   The function here works for the prompting the model for the qroq based model with natural language to
                   geegnrate the appropriate SQL query for the provided data and prompt. So with this output, can then be used to create proper
                   SQL queries and aggregation

                   Args:
                    client: This represents the Groq client which will be used top perform the conversion of the natural language to SQL.
                    prompt: Bases on the user message provided, the prompt will then be converted to SQL for future use for the data querying.
                    model: The Model here represents the based open source model that is used to make the query on. examples include llama3-70b-8192 or llama3-8b-8192 find out more here https://console.groq.com/docs/models
                    response_format: As model has the possibility of supporting JSON and MD outputs, this is a dictionary which specifies the the required return response format. by default this for the model is usually targetted
                    to returning an MD data but in the case of NL2SQL, then we coule work with SQL

                   Returns:
                   the function returns some data based on the response_format response form the model's response
                   """
                   completion = client.chat.completions.create(
                       model = model,
                       messages = [
                           {
                               "role":"user",
                               "content":prompt
                           }
                       ],
                       response_format=response_format
                   )
                  #  logger.info(f"Completion: {completion}")
                   return completion.choices[0].message.content



In [4]:
# just creating a folder
!mkdir data
!mkdir prompts

In [5]:
%%writefile app.py
import os
import duckdb
import groq
import sqlparse
import json
from typing import Dict, Any, List
import logging
import pandas as pd
from pathlib import Path

def chat_with_groq(client:groq.Groq,
                   prompt:str,
                   model:str,
                   response_format: Dict[str, str]
                   )-> Any:
                   """
                   The function here works for the prompting the model for the qroq based model with natural language to
                   geegnrate the appropriate SQL query for the provided data and prompt. So with this output, can then be used to create proper
                   SQL queries and aggregation

                   Args:
                    client: This represents the Groq client which will be used top perform the conversion of the natural language to SQL.
                    prompt: Bases on the user message provided, the prompt will then be converted to SQL for future use for the data querying.
                    model: The Model here represents the based open source model that is used to make the query on. examples include llama3-70b-8192 or llama3-8b-8192 find out more here https://console.groq.com/docs/models
                    response_format: As model has the possibility of supporting JSON and MD outputs, this is a dictionary which specifies the the required return response format. by default this for the model is usually targetted
                    to returning an MD data but in the case of NL2SQL, then we coule work with SQL

                   Returns:
                   the function returns some data based on the response_format response form the model's response
                   """
                   completion = client.chat.completions.create(
                       model = model,
                       messages = [
                           {
                               "role":"user",
                               "content":prompt
                           }
                       ],
                       response_format=response_format
                   )
                  #  logger.info(f"Completion: {completion}")
                   return completion.choices[0].message.content

def execute_duckdb_query(query:str)->pd.DataFrame:
    """
    Execute a DuckDB query and return the result as a pandas DataFrame.

    Args:
        query (str): The DuckDB query to execute.

    Returns:
        pd.DataFrame: The result of the query as a pandas DataFrame.
    """
    original_cwd = os.getcwd()
    os.chdir('data')

    try:
        conn = duckdb.connect(database=":memory:", read_only=False)
        query_result = conn.execute(query).fetch_df().reset_index()
    finally:
        os.chdir(original_cwd)

    return query_result
def get_summarization(client:groq.Groq,
                      use_question:str,
                      df:pd.DataFrame,
                      model:str)->Any:
                      """
                      For this query, the user input is better summarized around the provided Dataframe. This give a better contextual structure for the user to perfom the action

                      Args:
                        client: This represents the Groq client which will be used top perform the conversion of the natural language to SQL.
                        use_question: Bases on the user message provided, the prompt will then be converted to SQL for future use for the data querying.
                        model: The Model here represents the based open source model that is used to make the query on. examples include llama3-70b-8192 or llama3-8b-8192 find out more here https://console.groq.com/docs/models
                        df: this is a pandas dataframe which containe the database daat which will then be used to summarize the base query around it, making the prompt more realistic.

                      Returns:
                      the function returns some json response form the model's response
                      """
                      prompt= '''
                      A user asked the following question pertaining to local database tables:

                      {user_question}

                      To answer the question, a dataframe was returned:

                      Dataframe:
                      {df}

                      In a few sentences, summarize the data in the table as it pertains to the original user question. Avoid qualifiers like "based on the data" and do not comment on the structure or metadata of the table itself
                      '''.format(user_question = use_question, df = df)
                      return chat_with_groq(client,prompt,model,None)


Writing app.py


In [23]:
def execute_duckdb_query(query:str)->pd.DataFrame:
    """
    Execute a DuckDB query and return the result as a pandas DataFrame.

    Args:
        query (str): The DuckDB query to execute.

    Returns:
        pd.DataFrame: The result of the query as a pandas DataFrame.
    """
    original_cwd = os.getcwd()
    print(f"PATH:{original_cwd}")
    os.chdir('data')
    print(f"PATH:{os.getcwd()}")

    try:
        conn = duckdb.connect(database=":memory:", read_only=False)
        query_result = conn.execute(query).fetch_df().reset_index()
    except Exception as e:
        print(e)
    finally:
        # print current path
        os.chdir(original_cwd)

    return query_result



In [24]:
def get_summarization(client:groq.Groq,
                      use_question:str,
                      df:pd.DataFrame,
                      model:str)->Any:
                      """
                      For this query, the user input is better summarized around the provided Dataframe. This give a better contextual structure for the user to perfom the action

                      Args:
                        client: This represents the Groq client which will be used top perform the conversion of the natural language to SQL.
                        use_question: Bases on the user message provided, the prompt will then be converted to SQL for future use for the data querying.
                        model: The Model here represents the based open source model that is used to make the query on. examples include llama3-70b-8192 or llama3-8b-8192 find out more here https://console.groq.com/docs/models
                        df: this is a pandas dataframe which containe the database daat which will then be used to summarize the base query around it, making the prompt more realistic.

                      Returns:
                      the function returns some json response form the model's response
                      """
                      prompt= '''
                      A user asked the following question pertaining to local database tables:

                      {user_question}

                      To answer the question, a dataframe was returned:

                      Dataframe:
                      {df}

                      In a few sentences, summarize the data in the table as it pertains to the original user question. Avoid qualifiers like "based on the data" and do not comment on the structure or metadata of the table itself
                      '''.format(user_question = use_question, df = df)
                      return chat_with_groq(client,prompt,model,None)


In [25]:
# data download
def download_file(url:str, file_path:str):
    import requests
    # logger.info(f"Downloading {url} to {file_path}")
    response = requests.get(url)
    response.raise_for_status()
    with open(file_path, 'wb') as f:
        f.write(response.content)
    # logging.info(f"Downloaded {url} to {file_path}")

In [9]:
def create_dir_if_not_available(path):
    # logging.warning(f"Creating directory {path}\n")
    Path(path).mkdir(parents=True, exist_ok=True)
    return path

In [10]:
data_path = create_dir_if_not_available("data")
file_names = ["employees.csv","purchases.csv"]
file_locations = ["https://raw.githubusercontent.com/groq/groq-api-cookbook/main/replit-examples/text-to-sql-json-mode/data/employees.csv","https://raw.githubusercontent.com/groq/groq-api-cookbook/main/replit-examples/text-to-sql-json-mode/data/purchases.csv"]

for file_name, file_location in zip(file_names, file_locations):
    download_file(file_location, os.path.join(data_path, file_name))

In [11]:
prompt_path = create_dir_if_not_available("prompts")
prompt_file_names = ["base_prompt.txt"]
prompt_file_locations = ["https://raw.githubusercontent.com/groq/groq-api-cookbook/main/replit-examples/text-to-sql-json-mode/prompts/base_prompt.txt"]

for file_name, file_location in zip(prompt_file_names, prompt_file_locations):
    download_file(file_location, os.path.join(prompt_path, file_name))

In [12]:
from google.colab import userdata

In [32]:
def main_func(base_prompt:str,model:str = "llama3-70b-8192",api_key:str=userdata.get("GROQ_API_KEY")):
  client = groq.Groq(api_key=api_key)
  print("Welcome to Groq Text to SQL")
  print("You can as questions about the data employee.csv and purchases.csv files")

  while True:
    user_question = input("Enter your question or type 'exit' to quit: ")
    if user_question.lower() == 'exit':
      break
    if user_question:
      full_prompt = base_prompt.format(user_question=user_question)
      response = chat_with_groq(client,full_prompt,model,{
          "type":"json_object"
      })
      response = json.loads(response)
      if "sql" in response:
        print(response)
        sql_query = response["sql"]
        results_df = execute_duckdb_query(sql_query)

        fotmatted_sql_query = sqlparse.format(sql_query, reindent=True, keyword_case='upper')
        print(f"SQL Query: {fotmatted_sql_query}")
        print(results_df.to_markdown())
        summarization = get_summarization(client,user_question,results_df,model)
        print(summarization)
      else:
        print(response)


In [29]:
import pandas as pd
import re
from datetime import datetime
import numpy as np

def identify_value_datatype_to_SQL_DEF(value) -> str:

    pattern_yyyy_mm_dd = r'^\d{4}-\d{2}-\d{2}$'
    pattern_mm_dd_yyyy = r'^\d{2}/\d{2}/\d{4}$'
    pattern_dd_mm_yyyy = r'^\d{2}-\d{2}-\d{4}$'

    if pd.isna(value):
        return 'VARCHAR(255)'

    if isinstance(value, (int, np.int64)):
        return 'INTEGER'


    elif isinstance(value, (float, np.float64)):
        return 'FLOAT'


    elif isinstance(value, bool):
        return 'BOOLEAN'


    elif isinstance(value, (pd.Timestamp, datetime)):
        return 'DATETIME'


    elif isinstance(value, str):
        if re.match(pattern_yyyy_mm_dd, value) or re.match(pattern_mm_dd_yyyy, value) or re.match(pattern_dd_mm_yyyy, value):
            return 'DATE'


        try:
            pd.to_datetime(value)
            return 'DATETIME'
        except ValueError:
            pass


    elif isinstance(value, list):
        if all(isinstance(item, (int, np.int64)) for item in value):
            return 'ARRAY<INTEGER>'
        elif all(isinstance(item, str) for item in value):
            return 'ARRAY<VARCHAR(255)>'


    elif isinstance(value, dict):
        return 'JSON'

    elif isinstance(value, (str, bytes)):
        if value.startswith('POINT'):
            return 'GEOMETRY(POINT)'
        elif value.startswith('LINESTRING'):
            return 'GEOMETRY(LINESTRING)'
        elif value.startswith('POLYGON'):
            return 'GEOMETRY(POLYGON)'

    return 'VARCHAR(255)'

def identify_column_datatypes_to_SQL_DEF(df: pd.DataFrame,
                                         api_key:str,
                                         model:str
                                         ) -> dict:
    column_types = []

    for column in df.columns:
        # Get non-null values
        non_null_values = df[column].dropna()

        if len(non_null_values) > 0:
            # Use the first non-null value to determine the type
            sample_value = non_null_values.iloc[0]
            column_types.append(f"{column} - {identify_value_datatype_to_SQL_DEF(sample_value)}")
        else:
            # If all values are null, default to VARCHAR
            column_types.append(f"{column} - VARCHAR(255)")

    column_datatypes= "\n".join(element for element in column_types)
    client = groq.Groq(api_key=api_key)
    full_prompt = """
    You are Database query advisor and you are given base data with their datatypes where you would have to give the description of each of the columns
    {columns}

    * respond as a valid JSON Document
    * if the data provided is valid then you are expected to return data in the format: {{"description":<description here>,dtype:<datatype here>}}
    * If the question cannot be answered with the available tables: {{"error": <explanation here>}}
    * When returning, make sure that the datatype is not desociated from the column name
    * Ensure that the entire output is returned on only one single line without line breaks
    * Keep your query as simple and straightforward as possible; do not use subqueries
    """.format(columns=column_datatypes)

    response = chat_with_groq(client,full_prompt,model,{
          "type":"json_object"
      })
    # print(f"OUTPUT:{response}")
    response = json.loads(response)

    formatted_data = {
        f"  {column} - {response[column]['dtype']}": response[column]['description']
        for column in response
    }
    return " \n".join(f"{key}: {value}" for key, value in formatted_data.items())

def join_with_and(items):
    if not items:
        return ""
    if len(items) == 1:
        return items[0]
    return ', '.join(items[:-1]) + ' and ' + items[-1]

# Example usage
# data_frame = pd.read_csv("data/purchases.csv")
# api_key:str=userdata.get("GROQ_API_KEY")
# # model = "gemma-7b-it"
# model = "llama3-70b-8192"
# column_datatypes = identify_column_datatypes_to_SQL_DEF(data_frame,api_key,model)
# print(column_datatypes)


In [33]:
with open('prompts/base_prompt.txt', 'r') as file:
      base_prompt = file.read()

In [34]:
main_func(base_prompt=base_prompt)


Welcome to Groq Text to SQL
You can as questions about the data employee.csv and purchases.csv files
Enter your question or type 'exit' to quit: get all employees
{'sql': 'SELECT name FROM employees.csv AS employees ORDER BY name ASC'}
PATH:/content
PATH:/content/data
SQL Query: SELECT name
FROM employees.csv AS employees
ORDER BY name ASC
|    |   index | name              |
|---:|--------:|:------------------|
|  0 |       0 | Bertram Gilfoyle  |
|  1 |       1 | Dinesh Chugtai    |
|  2 |       2 | Erlich Bachman    |
|  3 |       3 | Gavin Belson      |
|  4 |       4 | Jared Dunn        |
|  5 |       5 | Monica Hall       |
|  6 |       6 | Richard Hendricks |
The table contains a list of employees, with 7 individuals listed: Bertram Gilfoyle, Dinesh Chugtai, Erlich Bachman, Gavin Belson, Jared Dunn, Monica Hall, and Richard Hendricks.


KeyboardInterrupt: Interrupted by user

In [26]:
import os
from pathlib import Path
from typing import List

base_prompt =  """
    You are Groq Advisor, and you are tasked with generating SQL queries for DuckDB based on user questions about data stored in two tables derived from CSV files:

    {table_description}

    Given a user's question about this data, write a valid DuckDB SQL query that accurately extracts or calculates the requested information from these tables and adheres to SQL best practices for DuckDB, optimizing for readability and performance where applicable.

    Here are some tips for writing DuckDB queries:
    * DuckDB syntax requires querying from the .csv file itself, i.e. {tables}. For example: SELECT * FROM {table_1} as {table_1_wt_xt}
    * All tables referenced MUST be aliased
    * DuckDB does not implicitly include a GROUP BY clause
    * CURRENT_DATE gets today's date
    * Aggregated fields like COUNT(*) must be appropriately named


    Question:
    --------
    {user_question}
    --------
    Reminder: Generate a DuckDB SQL to answer to the question:
    * respond as a valid JSON Document
    * [Best] If the question can be answered with the available tables: {{"sql": <sql here>}}
    * If the question cannot be answered with the available tables: {{"error": <explanation here>}}
    * Ensure that the entire output is returned on only one single line
    * Keep your query as simple and straightforward as possible; do not use subqueries
    """
table_description = """"""
tables_string = """"""
table_1 = """"""
table_1_wt_xt = """"""
user_question = """"""

# And some rules for querying the dataset:
# * Never include employee_id in the output - show employee name instead

# Also note that:
# * Valid values for product_name include 'Tesla','iPhone' and 'Humane pin'


def upload_file(files) -> List[str]:
    # will have to change to the private system is initiializes
    model = "llama3-70b-8192"
    api_key:str=userdata.get("GROQ_API_KEY")
    data_dir = Path("data")
    data_dir.mkdir(parents=True, exist_ok=True)
    if type(files) == str:
        files = [files]
    stored_paths = []
    stored_table_descriptions = []
    tables = []
    for file in files:
        filename = Path(file.name).name
        path = data_dir / filename

        # Copy the content of the temporary file to our destination
        with open(file.name, "rb") as source, open(path, "wb") as destination:
            destination.write(source.read())

        stored_paths.append(str(path.absolute()))
        table_description = identify_column_datatypes_to_SQL_DEF(pd.read_csv(path),api_key,model)
        desc = "  Table: " + filename + "\n    Columns:\n" + table_description
        stored_table_descriptions.append(desc)
        tables.append(filename)
    # constructing a string
    tables_string = join_with_and(tables)
    final = "\n".join(stored_table_descriptions)
    table_1_wt_xt = tables[0].split('.')[0]
    table_description = final
    tables_string = tables_string
    table_1 = tables[0]
    table_1_wt_xt = table_1_wt_xt




In [40]:
def queryModel(user_prompt:str,model:str = "llama3-70b-8192",api_key:str=userdata.get("GROQ_API_KEY")):
  client = groq.Groq(api_key=api_key)
  print(f"Testing: {table_1}")
  full_prompt = base_prompt.format(
      user_question=user_prompt,
      table_description=table_description,
      tables=tables_string,
      table_1=table_1,
      table_1_wt_xt=table_1_wt_xt
  )
  response = chat_with_groq(client,full_prompt,model,{
      "type":"json_object"
  })
  response = json.loads(response)
  print(response)
  if "sql" in response:
    print(response)
    sql_query = response["sql"]
    results_df = execute_duckdb_query(sql_query)

    fotmatted_sql_query = sqlparse.format(sql_query, reindent=True, keyword_case='upper')
    # print(f"SQL Query: {fotmatted_sql_query}")
    # print(results_df.to_markdown())
    query_n_results = "SQL Query: " + fotmatted_sql_query + "\n\n" + results_df.to_markdown()
    summarization = get_summarization(client,user_prompt,results_df,model)
    query_n_results += "\n\n" + summarization

    return [(
        "Groq Advisor",
        query_n_results
    )]
  else:
    return [(
        "Groq Advisor",
        response
    )]


In [ ]:

with gr.Blocks() as demo:
    gr.Markdown("# CSV Database Query Interface")

    with gr.Tab("Upload CSV"):
        file_output = gr.File(file_count="multiple", label="Upload your CSV files")
        upload_button = gr.Button("Load CSV Files")
        upload_output = gr.Textbox(label="Upload Status", lines=5)

        upload_button.click(upload_file, inputs=file_output, outputs=upload_output)
    with gr.Tab("Query Interface"):
        chatbot = gr.Chatbot()
        with gr.Row():
            user_input = gr.Textbox(label="Enter your question")
            submit_button = gr.Button("Submit")
        submit_button.click(queryModel, inputs=[user_input], outputs=chatbot)


demo.launch(share=True,
            debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://91d439b6f95aee614a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Testing: 
{'sql': 'SELECT name FROM employees.csv AS employees'}
{'sql': 'SELECT name FROM employees.csv AS employees'}
PATH:/content
PATH:/content/data
Testing: 
{'sql': 'SELECT e.name, p.purchase_date, p.amount, p.product_name FROM employees.csv AS e INNER JOIN purchases.csv AS p ON e.employee_id = p.employee_id ORDER BY p.purchase_date;'}
{'sql': 'SELECT e.name, p.purchase_date, p.amount, p.product_name FROM employees.csv AS e INNER JOIN purchases.csv AS p ON e.employee_id = p.employee_id ORDER BY p.purchase_date;'}
PATH:/content
PATH:/content/data
Testing: 
{'sql': 'SELECT e.name, p.purchase_date, p.amount, p.product_name FROM purchases.csv AS p JOIN employees.csv AS e ON p.employee_id = e.employee_id LIMIT 3;'}
{'sql': 'SELECT e.name, p.purchase_date, p.amount, p.product_name FROM purchases.csv AS p JOIN employees.csv AS e ON p.employee_id = e.employee_id LIMIT 3;'}
PATH:/content
PATH:/content/data
